# MCP Map Server - Test Notebook

## ⚠️ IMPORTANT: How This Notebook Works

**This notebook starts TEMPORARY server instances!**

Each cell that connects to the MCP server uses:
```python
StdioServerParameters(command="python", args=["server_sse.py"])
```

This **launches a NEW server process** for that cell only. When the cell finishes, the server shuts down.

## Two Ways to Use This:

### Option 1: Persistent Server (RECOMMENDED for viewing in browser)
1. **In a terminal, run:** `source .venv/bin/activate && python server_sse.py`
2. **Keep that terminal running**
3. **Open browser to:** http://localhost:8081
4. **Use the cells below** to modify the map - you'll see changes in real-time!

### Option 2: Temporary Servers (Testing only)
- Just run the cells below
- Each cell starts its own server temporarily
- Good for testing MCP tools work
- **Bad for viewing in browser** (server disappears after each cell)

## Which option are you using?
- [ ] Option 1: I have `python server_sse.py` running in a terminal
- [ ] Option 2: I'm just testing the MCP tools work

In [1]:
# Cell 1: Import required libraries
import asyncio
import json
from mcp.client.stdio import stdio_client, StdioServerParameters
from mcp.client.session import ClientSession

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [ ]:
# Cell 2: Setup - Define session ID and verify server is running

import subprocess
import socket

SESSION_ID = "notebook-test-session"

print(f"Using session ID: {SESSION_ID}")
print()

# Check if port 8081 is in use (server running)
def check_port(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('localhost', port))
    sock.close()
    return result == 0

if check_port(8081):
    print("✅ Server is RUNNING on port 8081")
    print("   You can view the map at: http://localhost:8081")
    print()
    print("   To view YOUR session:")
    print("   1. Open http://localhost:8081 in browser")
    print("   2. Open DevTools Console (F12)")
    print(f"   3. Run: document.cookie = 'mcp_map_session={SESSION_ID}; path=/'; location.reload();")
else:
    print("⚠️  WARNING: No server detected on port 8081")
    print()
    print("   You have two options:")
    print()
    print("   Option A) Start server in terminal (RECOMMENDED):")
    print("      source .venv/bin/activate && python server_sse.py")
    print("      Then re-run this cell")
    print()
    print("   Option B) Let notebook cells start temporary servers")
    print("      - Cells below will work but server disappears after each cell")
    print("      - You won't be able to view the map in browser")

Using session ID: notebook-test-session

To view this session in your browser:
1. Open http://localhost:8081
2. Open DevTools Console (F12)
3. Run: document.cookie = 'mcp_map_session=notebook-test-session; path=/'; location.reload();


---

## Understanding the Cells Below

**Important:** Each cell below that calls MCP tools does this:

```python
server_params = StdioServerParameters(
    command="python",
    args=["server_sse.py"]
)
```

This **starts a temporary server** just for that cell!

### If You Have a Server Running in Terminal:
- The temporary server will ALSO try to start
- It can't bind to port 8081 (already taken)
- **BUT** the MCP stdio part works anyway
- Changes ARE saved to Redis
- Your browser map will update in real-time! 🎉

### If You DON'T Have a Server Running:
- Each cell starts its own temporary server
- Server shuts down when cell finishes
- Changes still go to Redis
- But browser won't show anything (no persistent HTTP server)

**Bottom line:** For best experience, **run `python server_sse.py` in a terminal first!**

---

In [5]:
# Cell 3: Connect to MCP server and list tools
async def list_available_tools():
    """Connect to the MCP server and list available tools"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    print("🔌 Connecting to MCP Map Server...")
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # List available tools
            tools = await session.list_tools()
            print(f"\n✓ Connected! Found {len(tools.tools)} tools:\n")
            
            for tool in tools.tools:
                print(f"  📌 {tool.name}")
                print(f"     {tool.description}")
                print()
            
            return tools

# Run the function
tools = await list_available_tools()

🔌 Connecting to MCP Map Server...

✓ Connected! Found 9 tools:

  📌 add_layer
     Add a new map layer for a specific session. Include session_id parameter.

  📌 remove_layer
     Remove a layer from a session's map

  📌 toggle_layer
     Toggle layer visibility in a session

  📌 list_layers
     List all layers in a session

  📌 filter_layer
     Apply MapLibre filter to a layer in a session

  📌 set_layer_paint
     Set paint properties for a layer in a session

  📌 reset_layer_style
     Reset layer styling in a session

  📌 get_map_state
     Get complete map state for a session

  📌 set_map_view
     Set map center/zoom for a session



In [8]:
# Cell 4: Add a raster layer (wetlands)
async def add_wetlands_layer():
    """Add a raster layer showing global wetlands"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("🌊 Adding wetlands layer...")
            result = await session.call_tool("add_layer", {
                "session_id": SESSION_ID,
                "id": "wetlands",
                "type": "raster",
                "source": {
                    "type": "raster",
                    "tiles": ["https://minio.carlboettiger.info/public-cog/glwd/glwd/{z}/{x}/{y}.png"],
                    "tileSize": 256,
                    "minzoom": 0,
                    "maxzoom": 8
                },
                "visible": True
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                print(f"✓ {response.get('message')}")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await add_wetlands_layer()
print(f"\nFull response: {json.dumps(result, indent=2)}")

🌊 Adding wetlands layer...
✓ Added layer 'wetlands' to session notebook-test-session

Full response: {
  "success": true,
  "message": "Added layer 'wetlands' to session notebook-test-session",
  "layer": {
    "id": "wetlands",
    "type": "raster",
    "visible": true,
    "source": {
      "type": "raster",
      "tiles": [
        "https://minio.carlboettiger.info/public-cog/glwd/glwd/{z}/{x}/{y}.png"
      ],
      "tileSize": 256,
      "minzoom": 0,
      "maxzoom": 8
    },
    "layers": [
      {
        "id": "wetlands",
        "type": "raster",
        "source": "wetlands"
      }
    ],
    "paint": {},
    "filter": null
  }
}


In [10]:
# Cell 4a: Add OpenStreetMap basemap (WORKING TEST)
async def add_osm_basemap():
    """Add OpenStreetMap raster basemap - this should actually work!"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("🗺️ Adding OpenStreetMap basemap...")
            result = await session.call_tool("add_layer", {
                "session_id": SESSION_ID,
                "id": "osm-basemap",
                "type": "raster",
                "source": {
                    "type": "raster",
                    "tiles": ["https://tile.openstreetmap.org/{z}/{x}/{y}.png"],
                    "tileSize": 256,
                    "attribution": "© OpenStreetMap contributors"
                },
                "visible": True
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                print(f"✓ {response.get('message')}")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await add_osm_basemap()
print(f"\nFull response: {json.dumps(result, indent=2)}")

🗺️ Adding OpenStreetMap basemap...
✓ Added layer 'osm-basemap' to session notebook-test-session

Full response: {
  "success": true,
  "message": "Added layer 'osm-basemap' to session notebook-test-session",
  "layer": {
    "id": "osm-basemap",
    "type": "raster",
    "visible": true,
    "source": {
      "type": "raster",
      "tiles": [
        "https://tile.openstreetmap.org/{z}/{x}/{y}.png"
      ],
      "tileSize": 256,
      "attribution": "\u00a9 OpenStreetMap contributors"
    },
    "layers": [
      {
        "id": "osm-basemap",
        "type": "raster",
        "source": "osm-basemap"
      }
    ],
    "paint": {},
    "filter": null
  }
}


In [ ]:
# Cell 5: Add a vector layer (protected areas)
async def add_protected_areas():
    """Add a vector layer showing WDPA protected areas"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("🏞️  Adding WDPA protected areas layer...")
            result = await session.call_tool("add_layer", {
                "session_id": SESSION_ID,
                "id": "wdpa",
                "type": "vector",
                "source": {
                    "type": "vector",
                    "url": "pmtiles://https://s3-west.nrp-nautilus.io/public-wdpa/WDPA_Dec2025.pmtiles"
                },
                "layers": [
                    {
                        "id": "wdpa-fill",
                        "type": "fill",
                        "source": "wdpa",
                        "source-layer": "default",
                        "paint": {
                            "fill-color": "#00ff00",
                            "fill-opacity": 0.3
                        }
                    },
                    {
                        "id": "wdpa-outline",
                        "type": "line",
                        "source": "wdpa",
                        "source-layer": "default",
                        "paint": {
                            "line-color": "#006600",
                            "line-width": 1
                        }
                    }
                ],
                "visible": True
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                print(f"✓ {response.get('message')}")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await add_protected_areas()
print(f"\nFull response: {json.dumps(result, indent=2)}")

In [ ]:
# Cell 6: Filter protected areas to show only IUCN Category II (National Parks)
async def filter_to_national_parks():
    """Filter WDPA to show only IUCN Category II (National Parks)"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("🔍 Filtering to show only National Parks (IUCN Category II)...")
            result = await session.call_tool("filter_layer", {
                "session_id": SESSION_ID,
                "layer_id": "wdpa-fill",
                "filter": ["==", "IUCN_CAT", "II"]
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                print(f"✓ {response.get('message')}")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await filter_to_national_parks()
print(f"\nFull response: {json.dumps(result, indent=2)}")

In [9]:
# Cell 7: Set map view to focus on the United States
async def set_us_view():
    """Set map view to show the United States"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("🗺️  Setting map view to United States...")
            result = await session.call_tool("set_map_view", {
                "session_id": SESSION_ID,
                "center": [-98.5795, 39.8283],  # Center of US
                "zoom": 4
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                print(f"✓ {response.get('message')}")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await set_us_view()
print(f"\nFull response: {json.dumps(result, indent=2)}")

🗺️  Setting map view to United States...
✓ None

Full response: {
  "success": true,
  "center": [
    -98.5795,
    39.8283
  ],
  "zoom": 4
}


In [ ]:
# Cell 8: List all layers in our session
async def list_session_layers():
    """List all layers currently in our session"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("📋 Listing all layers in session...")
            result = await session.call_tool("list_layers", {
                "session_id": SESSION_ID
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                layers = response.get('layers', [])
                print(f"\n✓ Found {len(layers)} layer(s):\n")
                for layer in layers:
                    visible = "✓" if layer.get('visible') else "✗"
                    print(f"  {visible} {layer['id']} ({layer['type']})")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await list_session_layers()
print(f"\nFull response: {json.dumps(result, indent=2)}")

## Viewing Your Map

To see the map with your changes:

1. Make sure the MCP server is running in the background or start it:
   ```bash
   python server_sse.py
   ```

2. Open your browser to: http://localhost:8081

3. Set the session cookie to match this notebook's session:
   - Open DevTools (F12)
   - Go to Console
   - Run: `document.cookie = 'mcp_map_session=notebook-test-session; path=/'; location.reload();`

4. Your map should now show:
   - Wetlands layer (raster)
   - WDPA protected areas filtered to IUCN Category II (National Parks)
   - Centered on the United States

## Next Steps

Try modifying the cells above to:

- Add different layers from examples.md
- Change the styling (colors, opacity)
- Apply different filters
- Change the map view to other locations
- Toggle layer visibility

See the [examples.md](examples.md) file for more ideas!